<h2>Market Study - Chicken Exports</h2>

<a href="https://openclassrooms.com/en/" >Openclassrooms</a>, Data Analyst Course
<br>Project 5 - Michael Orange


<hr>

A company in the food industry specialized in chicken is looking at expanding worldwide. All countries are considered. 
<br>The main objective is to provide the company with data ensuring it will collaborate with the most promising export market countries. 

Grouping countries in 'clusters' based on their similarity is required. 
<br>Default variables are: 
- population growth, 
- calory supply per capita and protein supply per capita, 
- prevalence of animal proteins in the mix. 

Additional relevant variable might be included (ex. GDP per capita)

<hr>

**Section 1** [Importing FAOSTAT Datasets](#import)


**Section 2** [Adding General information](#general)
- [Population](#pop)
- [Gross Domestic Product](#gdp)
- [Political Stability](#stab)
- [European Union countries](#eu)
    
    
**Section 3** [Adding Food-related data](#food)
- [Food-Balance](#fb)
- [Diet - Calories and Proteins](#diet)
- [Poulty data](#poultry)

**Section 4** [Imputating missing data](#imputation)
    
**Section 5** [Exporting Data Country](#export) 

<hr>

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from display_pca import *

<h2>Importing FAOSTAT Datasets</h2><a id='import'></a>

Data are collected from The Food and Agriculture Organization Corporate Statistical Database (FAOSTAT). The FAOSTAT website disseminates statistical data collected and maintained by the Food and Agriculture Organization (FAO).

FAOSTAT - Data collected (2013 and 2017 are selected for the below analysis): analysis is based on 2017 figures, with a comparaison from 2013 data. 
2017 is the most recent comprehensive data from FAOSTAT.

- Datasets Food Balance Animal and Vegetal
- Dataset Population
- Dataset Gross Domestic Product (GDP)
- Dataset Political Stability Index
- Dataset Trade Mix - Chicken 

In [2]:
#import food datasets
veg2013 = pd.read_csv("data/raw/vegetal2013.csv")
ani2013 = pd.read_csv("data/raw/animal2013.csv")
veg2017 = pd.read_csv('data/raw/vegetal2017.csv', dtype={'Note': str })
ani2017 = pd.read_csv("data/raw/animal2017.csv")
ani2013["origin"] = "animal"
veg2013["origin"] = "vegetal"
ani2017["origin"] = "animal"
veg2017["origin"] = "vegetal"

#import population
pop = pd.read_csv("data/raw/FAOSTAT_data_Population_2013-2017.csv")

# import GDP and Stability
gdp = pd.read_csv("data/raw/FAOSTAT_data_MacroIndicators_20132017.csv")
stability = pd.read_csv("data/raw/FAOSTAT_data_PoliticalStability_2017.csv")

#import food index
foodindex = pd.read_csv("data/raw/FAOSTAT_data_FoodIndex_20132017.csv")

# import EU countries
eu_country = pd.read_csv("data/raw/listofeucountries.csv")

**Important note** 
Naming convention: analysis is based on 2017 data, the default year is 2017 for the variables. 
- For the variables related to 2017, indication of the 'year' is not included. ex. 'pop_thousand' =population 2017.
- For the variables related to 2013, '2013' is always included - ex. 'pop_thousand_2013' =population 2013.

<hr>

<h2>Adding general information</h2><a id='general'></a>

<h3>Population</h3><a id='pop'></a>

Variables added to the dataset:
- Population per country
- Growth of the population (%) - 2017 vs 2013

In [3]:
# population
pop.columns = ["xx","xx2","country_code","country", 'xx3', 'xx4', 'xx5','xx6', 
               'xx7', 'year',"xx8","population_total_thousand","xx9", 'xx10', 'xx11']

pop = pop.drop(["xx","xx2",'xx3','xx4','xx5', 'xx6', 'xx7','xx8', 'xx9', 'xx10', 'xx11'], axis = 1)
pop.reset_index(drop=True, inplace=True)

data_country = pop.pivot_table(index=['country_code', 'country'], columns = ['year'], values=['population_total_thousand'])
data_country.columns = ['pop_thousand_2013', 'pop_thousand']
data_country.reset_index(inplace=True)

#growth population
data_country['pop_growth'] = (data_country['pop_thousand'] / data_country['pop_thousand_2013'] - 1) *100

<h3>Gross Domestic Product (GDP) per capita</h3><a id='gdp'></a>

- GDP per capita (USD)
- GDP Growth - 2017 vs 2013

In [4]:
gdp.columns = ["xx", "xx2", "country_code", "country", 'xx3', 'xx4', 'xx5', 'item', 'xx6', "year", "xx7", "value", 'xx8', 'xx9', 'xx10']
gdp = gdp.loc[gdp['item'] == 'Gross Domestic Product per capita'].pivot_table(\
                                                                                                      index=['country_code', "country"], columns = ['year', 'item'], values=['value'])
gdp.columns = ['gdp_percapita_usd_2013', 'gdp_percapita_usd']
gdp.reset_index(inplace=True)

data_country = pd.merge(data_country, gdp, how='left')

GDP of Taiwan is missing. 
Imputation with information collected from the International Monetary Fund (IMF).
- Taiwan GDP per capita 2013 : 43 831 USD 
- Taiwan GDP per capita 2017 : 50 593 USD 
Source: https://bit.ly/2U2251b

In [5]:
# Imputation GDP Taiwan
data_country.loc[data_country['country'] == 'China, Taiwan Province of', 'gdp_percapita_usd_2013'] = 43831
data_country.loc[data_country['country'] == 'China, Taiwan Province of', 'gdp_percapita_usd'] = 50593

In [6]:
# Growth GDP
data_country['gdp_growth'] = np.where(data_country['gdp_percapita_usd_2013'] != 0,
                                                             (data_country['gdp_percapita_usd'] / data_country['gdp_percapita_usd_2013'] - 1) * 100, 0)

In [7]:
#remove column not needed for the analysis
data_country.drop(['gdp_percapita_usd_2013'], axis = 1, inplace=True)

<h3>Political Stability</h3><a id='stab'></a>

In [8]:
stability.columns = ["xx", "xx2", "country_code", "country", 'xx3', 'xx4', 'xx5', 'item', 'xx6', "xx8", "xx8", "value", 'xx9', 'xx10', 'xx11']
stability = stability.pivot_table(index=['country_code', 'country'], columns = ['item'], values=['value'])
stability.columns = ['political_stability_index']
stability.reset_index(inplace=True)

data_country = pd.merge(data_country, stability, how='left')

<h3>European Union countries</h3><a id='eu'></a>

In [9]:
#countries with different spellings between eu_country and data_country
eu_country.loc[~eu_country['x'].isin(data_country['country'])]

,x
5,Czech Republic
23,Slovak Republic
27,United Kingdom


In [10]:
# correct names
eu_country.loc[eu_country['x'] == 'Czech Republic', 'x'] = 'Czechia'
eu_country.loc[eu_country['x'] == 'Slovak Republic', 'x'] = 'Slovakia'
eu_country.loc[eu_country['x'] == 'United Kingdom', 'x'] = 'United Kingdom of Great Britain and Northern Ireland'

In [11]:
# flag EU countries
data_country.loc[data_country['country'].isin(eu_country['x']), 'euro_union'] = 'EU'
data_country.loc[~data_country['country'].isin(eu_country['x']), 'euro_union'] = 'Outside EU'

<hr>

<h2>Adding Food-related data</h2><a id='food'></a>

<h3>Food Balance</h3><a id='fb'></a>

In [12]:
# group food datatsets
temp = [veg2013, ani2013, ani2017, veg2017]
temp = pd.concat(temp, ignore_index=True)

# delete ani2013, veg2013, ani2017, veg2017
del ani2013, veg2013, ani2017, veg2017

temp.columns = ["xx", "xx2", "country_code", "country", 'xx3', 'element', 'item_code', 'item',
                'xx4', "year", "unit", "value", 'xx5', 'flag_description', 'origin', 'xx6']

data = temp.pivot_table(index=["year", "country_code", "country", "origin", "item_code", \
                               "item"], columns = ["element"], values=["value"], aggfunc=sum)

# rename columns
data.columns = ['domestic_supply_quantity','export_quantity','fat_supply_quantity_gcapitaday','feed',
                'food','food_supply_kcalcapitaday','food_supply_quantity_kgcapitayr','import_quantity','losses','other_uses','processing',
                'production', 'protein_supply_quantity_gcapitaday', 'residuals', 'seed','stock_variation','tourist_consumption']

data = data.reset_index()

# merge data and pop
data = pd.merge(data, pop, how='left')

Data for China are duplicated. 
<br>data for 'China' = sum of the splitted data for 'China, mainland', 'China, Hong Kong', 'China, Macao', 'China, Province of Taiwan'.
- deletion of 'China' data (code 351)
- 'China, mainland', 'China, Hong Kong', 'China, Macao', 'China, Province of Taiwan' data are kept in order to preserve a better granularity.


Data for Bermuda and Brunei are no longer available in 2017.

In [13]:
# remove doublon from China (code_country 351)
data = data.loc[data.country_code != 351]

# remove Bermuda and Brunei - no information for 2017
data = data.loc[data.country != 'Bermuda']
data = data.loc[data.country != 'Brunei Darussalam']

<h3>Diet - Calories and Proteins</h3><a id='diet'></a>

- Food supply kcal per capita per day - per country
- Protein supply g per capita per day - per country

In [14]:
data.columns

Index(['year', 'country_code', 'country', 'origin', 'item_code', 'item',
       'domestic_supply_quantity', 'export_quantity',
       'fat_supply_quantity_gcapitaday', 'feed', 'food',
       'food_supply_kcalcapitaday', 'food_supply_quantity_kgcapitayr',
       'import_quantity', 'losses', 'other_uses', 'processing', 'production',
       'protein_supply_quantity_gcapitaday', 'residuals', 'seed',
       'stock_variation', 'tourist_consumption', 'population_total_thousand'],
      dtype='object')

In [15]:
temp = data.pivot_table(index=['country_code', 'country'], columns=['year'],
                        values=['food_supply_kcalcapitaday',  'protein_supply_quantity_gcapitaday', 'domestic_supply_quantity'], aggfunc=sum)
temp.columns = ['food_supply_kcalcapitaday_2013', 'protein_supply_gcapitaday_2013', 'food_supply_kcalcapitaday', 'protein_supply_gcapitaday',  
                'dom_supply_tons_2013', 'dom_supply_tons']
temp = temp.reset_index()

data_country = pd.merge(temp, data_country, how='left')

**Animal proteins in the total protein supply**

- Prevalence of the animal proteins in the total proteins - per country
- Growth of the protein supply per country (%) from animals - 2017 vs 2013

In [16]:
NB_DAYS_YEAR = 365 

In [17]:
# Protein Supply in kg
for (x,y, z) in [('protein_supply_gcapitaday','pop_thousand', 'protein_supply_kg'),
              ('protein_supply_gcapitaday_2013','pop_thousand_2013', 'protein_supply_kg_2013') ]:
    
    data_country[z] = data_country[x] / 1000 * NB_DAYS_YEAR *  data_country[y] * 1000

In [18]:
temp = data.loc[data['origin']=='animal'].pivot_table(index=['country_code', 'country'], columns=['year'],
                                                      values=[ 'protein_supply_quantity_gcapitaday'], aggfunc=sum)
temp.columns = ['protein_supply_animal_gcapitaday_2013', 'protein_supply_animal_gcapitaday']
temp.reset_index(inplace=True)

data_country = pd.merge(data_country, temp, how='left')

In [19]:
# Protein Supply from animals in kg 
for (x,y, z) in [('protein_supply_animal_gcapitaday','pop_thousand', 'protein_supply_animal_kg'),
              ('protein_supply_animal_gcapitaday_2013','pop_thousand_2013', 'protein_supply_animal_kg_2013') ]:
    
    data_country[z] = data_country[x] / 1000 * NB_DAYS_YEAR *  data_country[y] * 1000

In [20]:
# Proportion animal proteins in the total protein supply
for (x,y, z) in [('protein_supply_animal_kg','protein_supply_kg', 'protein_animal_over_protein'),
              ('protein_supply_animal_kg_2013','protein_supply_kg_2013', 'protein_animal_over_protein_2013'), 
                ]:
    
    data_country[z] = data_country[x] / data_country[y] * 100

In [21]:
#remove column not needed for the analysis
data_country.drop(['food_supply_kcalcapitaday_2013' , 'protein_supply_gcapitaday_2013', 
       'pop_thousand_2013', 
       'protein_supply_kg', 'protein_supply_kg_2013',
       'protein_supply_animal_gcapitaday_2013',
       'protein_supply_animal_gcapitaday', 'protein_supply_animal_kg',
       'protein_supply_animal_kg_2013', 
       'protein_animal_over_protein_2013',  'dom_supply_tons_2013'], axis = 1, inplace=True)

<h3>Poultry key data</h3><a id='poultry'></a>

Poultry are domesticated avian species that can be raised for eggs, meat and/or feathers. The term “poultry” covers a wide range of birds, from indigenous and commercial breeds of chickens to Muscovy ducks, mallard ducks, turkeys, guinea fowl, geese, quail, pigeons, ostriches and pheasants. 

- In 2017, chickens accounted for some 92 percent of the world’s poultry population, followed by ducks (5 percent), and turkeys (2 percent). 
- Chickens contribute 89 percent of world poultry meat production, followed by turkeys with 5 percent, ducks with 4 percent and geese and guinea fowl with 2 percent. The rest comes from other poultry species.
- Chickens provide 92 percent of world egg production.

source: http://www.fao.org/poultry-production-products/production/en/

There is no granular data about chicken exclusively in the dataset 'animal' but the data for the category 'Poultry' (in which chicken is accounting around 90% of the total) give us a fair indicator of the chicken market in each country. 

Poultry composition: 
- Meat chicken, Fat liver prepared (foie gras), Meat chicken canned, Meat,duck, Meat goose and guinea fowl, Meat turkey.

In [22]:
data.columns

Index(['year', 'country_code', 'country', 'origin', 'item_code', 'item',
       'domestic_supply_quantity', 'export_quantity',
       'fat_supply_quantity_gcapitaday', 'feed', 'food',
       'food_supply_kcalcapitaday', 'food_supply_quantity_kgcapitayr',
       'import_quantity', 'losses', 'other_uses', 'processing', 'production',
       'protein_supply_quantity_gcapitaday', 'residuals', 'seed',
       'stock_variation', 'tourist_consumption', 'population_total_thousand'],
      dtype='object')

In [23]:
temp = data.loc[data['item'] == 'Poultry Meat'].pivot_table(index=['country_code', 'country'], columns=['year'], \
                                                            values=[ 'domestic_supply_quantity', 'production', 'import_quantity', 'export_quantity', 'protein_supply_quantity_gcapitaday'], aggfunc=sum)

temp.columns = ['dom_supply_poultry_tons_2013', 'dom_supply_poultry_tons', 
                'prod_poultry_tons_2013', 'prod_poultry_tons', 
                'import_poultry_tons_2013', 'import_poultry_tons', 
                'export_poultry_tons_2013', 'export_poultry_tons',
                'protein_poultry_gcapitaday_2013', 'protein_poultry_gcapitaday']

temp.reset_index(inplace=True)

#convert 1000 tons to tons
for z in temp.iloc[:, list(range(2, len(temp.columns)-2))].columns:
    temp[str(z)] = temp[str(z)] * 1000

data_country = pd.merge(data_country, temp, how='left')

- % growth Poulty Imports (2017 vs 2013)
- % growth Poultry Production (2017 vs 2013)



In [24]:
 data_country['net_import_poultry_tons'] = data_country['import_poultry_tons'] - data_country['export_poultry_tons']
data_country['net_import_poultry_tons_2013'] = data_country['import_poultry_tons_2013'] - data_country['export_poultry_tons_2013']

In [25]:
#Poultry: Proportion of imports of in the domestic supply, Proportion of production in the domestic supply

for (x,y, z) in [('dom_supply_poultry_tons','net_import_poultry_tons', 'net_import_poultry_over_domsupply'),
              ('dom_supply_poultry_tons','import_poultry_tons', 'import_poultry_over_domsupply'),
                 ('dom_supply_poultry_tons','prod_poultry_tons', 'prod_poultry_over_domsupply')]:
    data_country.loc[(data_country[x] ==0) & (data_country[y] ==0), z] = 0
    data_country.loc[(data_country[x] ==0) & (data_country[y] !=0), z] = 100
    data_country.loc[data_country[x] !=0, z] = (data_country[y] / data_country[x]) * 100


In [26]:
# Poultry:  growth production and growth import

for (x,y, z) in [('net_import_poultry_tons_2013','net_import_poultry_tons', 'growth_net_import_poultry')]:
    
    data_country.loc[(data_country[x] ==0) & (data_country[y] ==0), z] = 0
    data_country.loc[(data_country[x] ==0) & (data_country[y] !=0), z] = 100
    data_country.loc[data_country[x] !=0, z] = (data_country[y] / data_country[x] - 1) * 100

In [27]:
#remove column not needed for the analysis
data_country.drop(['dom_supply_poultry_tons_2013', 'prod_poultry_tons_2013', 'import_poultry_tons_2013', 'export_poultry_tons_2013', 
                   'export_poultry_tons', 'net_import_poultry_tons_2013', 'protein_poultry_gcapitaday_2013']
                  , axis = 1, inplace=True)

<hr>

<h2>Imputing missing data and verifying dataset</h2><a id='imputation'></a>

Countries with no political stability index: {{data_country.loc[data_country['political_stability_index'].isna()].country.tolist()}}

- Imputation possible for the 2 French territories (French Polynesia and New Caledonia) with France's index.
- Imputation possible for Namibia with United Arab Emirates' index - the political stability index of the UAE is the closest (and almost equivalent) of Namibia in 2017 (source: World Bank - https://www.theglobaleconomy.com/rankings/wb_political_stability/)

In [28]:
# imputation French Polynesia and New Caledonia
for z in ['French Polynesia', 'New Caledonia']:
    data_country.loc[data_country['country'] == z, 'political_stability_index'] = \
                                            data_country.loc[data_country['country'] == 'France']['political_stability_index'].values

# imputation Namibia
data_country.loc[data_country['country'] == 'Namibia', 'political_stability_index'] = \
                                            data_country.loc[data_country['country'] == 'United Arab Emirates']['political_stability_index'].values

In [29]:
data_country.isna().any()

country_code                         False
country                              False
food_supply_kcalcapitaday            False
protein_supply_gcapitaday            False
dom_supply_tons                      False
pop_thousand                         False
pop_growth                           False
gdp_percapita_usd                    False
gdp_growth                           False
political_stability_index            False
euro_union                           False
protein_animal_over_protein          False
dom_supply_poultry_tons              False
prod_poultry_tons                    False
import_poultry_tons                  False
protein_poultry_gcapitaday           False
net_import_poultry_tons              False
net_import_poultry_over_domsupply    False
import_poultry_over_domsupply        False
prod_poultry_over_domsupply          False
growth_net_import_poultry            False
growth_domsupply_poultry             False
dtype: bool

<h2>Preparation datasets</h2><a id='prep'></a>

In [30]:
#remove column not needed for the analysis
data_country.drop(['country_code'], axis = 1, inplace=True)

<h3>Simplified dataset</h3><a id='simplified'></a>

In [31]:
data_country.columns

Index(['country', 'food_supply_kcalcapitaday', 'protein_supply_gcapitaday',
       'dom_supply_tons', 'pop_thousand', 'pop_growth', 'gdp_percapita_usd',
       'gdp_growth', 'political_stability_index', 'euro_union',
       'protein_animal_over_protein', 'dom_supply_poultry_tons',
       'prod_poultry_tons', 'import_poultry_tons',
       'protein_poultry_gcapitaday', 'net_import_poultry_tons',
       'net_import_poultry_over_domsupply', 'import_poultry_over_domsupply',
       'prod_poultry_over_domsupply', 'growth_net_import_poultry',
       'growth_domsupply_poultry'],
      dtype='object')

In [32]:
data_country_simplified = data_country[['country', 'food_supply_kcalcapitaday', 'pop_growth', 'protein_supply_gcapitaday',
                                        'protein_animal_over_protein', 'pop_thousand', 'euro_union']]

In [33]:
data_country_simplified

,country,food_supply_kcalcapitaday,pop_growth,protein_supply_gcapitaday,protein_animal_over_protein,pop_thousand,euro_union
0,Armenia,2924.0,1.629045,3072.0,1.450521,2944.791,Outside EU
1,Afghanistan,2087.0,12.477767,1997.0,0.528793,36296.113,Outside EU
2,Albania,3188.0,-0.675703,3400.0,1.950588,2884.169,Outside EU
3,Algeria,3293.0,8.518733,3345.0,0.768311,41389.189,Outside EU
4,Angola,2474.0,14.610305,2266.0,0.726831,29816.766,Outside EU
...,...,...,...,...,...,...,...
167,Belgium,3737.0,2.382453,3770.0,1.535279,11419.748,EU
168,Luxembourg,3540.0,9.095768,3334.0,2.020996,591.910,EU
169,Serbia,2725.0,-1.000975,2799.0,1.429439,8829.628,Outside EU
170,Montenegro,3490.0,0.194462,3478.0,1.974411,627.563,Outside EU


<h3>Complete dataset</h3><a id='simplified'></a>

data_country : complete dataset
- data_pca : data_country with only the variable required fo the PCA
    - data_pca_poultry : sub dataset of data_pca with only variables related to Poultry 
    - data_pca_not_poultry : sub dataset of data_pca with variables not related to Poultry

In [34]:
data_country

,country,food_supply_kcalcapitaday,protein_supply_gcapitaday,dom_supply_tons,pop_thousand,pop_growth,gdp_percapita_usd,gdp_growth,political_stability_index,euro_union,...,dom_supply_poultry_tons,prod_poultry_tons,import_poultry_tons,protein_poultry_gcapitaday,net_import_poultry_tons,net_import_poultry_over_domsupply,import_poultry_over_domsupply,prod_poultry_over_domsupply,growth_net_import_poultry,growth_domsupply_poultry
0,Armenia,2924.0,3072.0,97.33,2944.791,1.629045,3078.978564,9.889928,-0.71,Outside EU,...,47000.0,0.0,35000.0,5.44,24000.0,51.063830,74.468085,0.000000,0.000000,17.500000
1,Afghanistan,2087.0,1997.0,54.09,36296.113,12.477767,468.297893,-5.206857,-2.78,Outside EU,...,57000.0,0.0,29000.0,0.54,1000.0,1.754386,50.877193,0.000000,-95.454545,-24.000000
2,Albania,3188.0,3400.0,119.50,2884.169,-0.675703,3347.701760,7.818559,0.40,Outside EU,...,47000.0,0.0,38000.0,6.26,25000.0,53.191489,80.851064,0.000000,212.500000,11.904762
3,Algeria,3293.0,3345.0,92.85,41389.189,8.518733,3264.338962,-7.348644,-0.96,Outside EU,...,277000.0,0.0,2000.0,1.97,-273000.0,-98.555957,0.722022,0.000000,-4.210526,-4.810997
4,Angola,2474.0,2266.0,54.09,29816.766,14.610305,2805.692595,-15.509934,-0.29,Outside EU,...,319000.0,0.0,277000.0,3.60,235000.0,73.667712,86.833856,0.000000,-24.679487,-14.247312
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167,Belgium,3737.0,3770.0,101.35,11419.748,2.382453,32988.016625,0.652140,0.42,EU,...,152000.0,656000.0,338000.0,4.57,-125000.0,-82.236842,222.368421,431.578947,-8.759124,6.293706
168,Luxembourg,3540.0,3334.0,105.79,591.910,9.095768,81059.319125,3.178482,1.34,EU,...,11000.0,1000.0,11000.0,7.19,11000.0,100.000000,100.000000,9.090909,37.500000,0.000000
169,Serbia,2725.0,2799.0,82.43,8829.628,-1.000975,4580.333116,6.113513,0.10,Outside EU,...,90000.0,7000.0,12000.0,3.50,-73000.0,-81.111111,13.333333,7.777778,-9.876543,-10.891089
170,Montenegro,3490.0,3478.0,113.12,627.563,0.194462,5609.035115,11.469796,0.01,Outside EU,...,10000.0,0.0,8000.0,5.79,4000.0,40.000000,80.000000,0.000000,0.000000,0.000000


In [35]:
data_country.columns

Index(['country', 'food_supply_kcalcapitaday', 'protein_supply_gcapitaday',
       'dom_supply_tons', 'pop_thousand', 'pop_growth', 'gdp_percapita_usd',
       'gdp_growth', 'political_stability_index', 'euro_union',
       'protein_animal_over_protein', 'dom_supply_poultry_tons',
       'prod_poultry_tons', 'import_poultry_tons',
       'protein_poultry_gcapitaday', 'net_import_poultry_tons',
       'net_import_poultry_over_domsupply', 'import_poultry_over_domsupply',
       'prod_poultry_over_domsupply', 'growth_net_import_poultry',
       'growth_domsupply_poultry'],
      dtype='object')

In [36]:
# variables to be used on the PCA 
data_pca = data_country[['country', 'pop_growth', 'gdp_percapita_usd',  'political_stability_index',
        'food_supply_kcalcapitaday', 'protein_supply_gcapitaday',
        'protein_animal_over_protein',
       'protein_poultry_gcapitaday',
        'prod_poultry_over_domsupply', 'net_import_poultry_over_domsupply',
        'growth_net_import_poultry']]

In [37]:
data_pca.describe()

,pop_growth,gdp_percapita_usd,political_stability_index,food_supply_kcalcapitaday,protein_supply_gcapitaday,protein_animal_over_protein,protein_poultry_gcapitaday,prod_poultry_over_domsupply,net_import_poultry_over_domsupply,growth_net_import_poultry
count,172.000000,172.000000,172.000000,172.000000,172.000000,172.000000,172.000000,172.000000,172.000000,172.000000
mean,5.475461,10567.131253,-0.060000,2847.610465,2858.395349,1.239316,7.121279,14.441610,-30.935892,29.417984
std,5.016197,14523.104601,0.895993,440.765024,454.544301,0.576089,5.603464,49.683412,86.337931,94.618434
min,-5.434483,287.166959,-2.960000,1879.000000,1754.000000,0.253906,0.040000,0.000000,-198.615917,-300.000000
25%,1.910891,1588.501797,-0.612500,2523.250000,2514.250000,0.746637,2.165000,0.000000,-97.208532,-1.046760
50%,4.755661,4467.101922,0.030000,2830.500000,2871.500000,1.229501,6.505000,0.000000,-62.107219,14.185111
75%,9.093107,12088.189295,0.667500,3196.500000,3250.250000,1.667150,10.365000,9.090909,42.924848,37.500000
max,23.935417,81059.319125,1.590000,3770.000000,3770.000000,3.015508,27.870000,431.578947,315.357143,500.000000


In [38]:
# subset of data_pca with only data related to Poultry and Chicken 
data_pca_poultry = data_country[['country', 'protein_poultry_gcapitaday',
         'prod_poultry_over_domsupply', 'net_import_poultry_over_domsupply',
        'growth_net_import_poultry']]

In [39]:
# subset of data_pca with data NOT related to Poultry and Chicken 
data_pca_not_poultry = data_pca.copy()
for i in data_pca.columns: 
    if i in data_pca_poultry.columns and i != 'country' and i!= 'import_poultry_over_domsupply':
        data_pca_not_poultry.drop([i], axis = 1, inplace=True)

<hr>

<h2>Exporting Dataset data_country</h2><a id='export'></a>

<h3>Export simplified dataset</h3>

In [40]:
data_country_simplified.to_csv(r'data/output/data_country_simplified.csv', index = False)

<h3>Export complete dataset</h3>

In [41]:
data_country.to_csv(r'data/output/data_country.csv', index = False)

In [42]:
data_pca.to_csv(r'data/output/data_pca.csv', index = False)

In [43]:
data_pca_poultry.to_csv(r'data/output/data_pca_poultry.csv', index = False)

In [44]:
data_pca_not_poultry.to_csv(r'data/output/data_pca_not_poultry.csv', index = False)